## DCGAN implementation in Keras

Inspired by jacobgil's implementation: [https://github.com/jacobgil/keras-dcgan/](https://github.com/jacobgil/keras-dcgan)

In [3]:
# import cPickle as pickle

# with open('shrunk_baroque.pkl','rb') as f:
#     imgs = pickle.load(f)
# imgs.shape

(3226, 120, 120, 3)

In [2]:
from __future__ import print_function,division
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Reshape
from keras.layers.core import Activation
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import UpSampling2D
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.layers.core import Flatten
from keras.optimizers import SGD
from keras.datasets import mnist
from tqdm import tqdm_notebook 
import numpy as np

%pylab inline

Using TensorFlow backend.

Populating the interactive namespace from numpy and matplotlib


pylab import has clobbered these variables: ['f']
`%matplotlib` prevents importing * from pylab and numpy


## Models

In [3]:
# TODO(jkg): add better descriptions of models

def keras_generator_model():
    '''generate images
    '''
    model = Sequential()
    ##########
    model.add(Dense(input_dim = 100, output_dim = 1024))
    model.add(Activation('tanh'))
    ##########
    model.add(Dense(128*7*7))
    model.add(BatchNormalization())
    model.add(Activation('tanh'))
    ##########
    model.add(Reshape((128,7,7),input_shape=(128*7*7,)))
    model.add(UpSampling2D(size=(2,2)))
    model.add(Convolution2D(64,5,5,border_mode = 'same'))
    model.add(Activation('tanh'))
    ##########
    model.add(UpSampling2D(size=(2,2)))
    model.add(Convolution2D(1,5,5,border_mode='same'))
    model.add(Activation('tanh'))
    return model

def keras_discriminator_model():
    '''discriminate images
    '''
    model = Sequential()
    model.add(Convolution2D(64, 5, 5,
                           border_mode = 'same',
                           input_shape = (1,28,28)))
    model.add(Activation('tanh'))
    ##########
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Convolution2D(128, 5, 5))
    model.add(Activation('tanh'))
    ##########
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Flatten())
    ##########
    model.add(Dense(1024))
    model.add(Activation('tanh'))
    ##########
    model.add(Dense(1))
    model.add(Activation('sigmoid'))
    return model

def generator_and_discriminator(gen_model,dis_model):
    '''stack the generator model on top of the discriminator model
    only allow the generator model to be trainable
    '''
    model = Sequential()
    model.add(gen_model)
    dis_model.trainable = False
    model.add(dis_model)
    return model

## Testing model w/ MNIST

In [4]:
# Normalize and Reshape data
(X_train,y_train),(X_test,y_test) = mnist.load_data()

# why do this instead of convert to [0,1] - center and scale not just scale
X_train_n = (X_train.astype(np.float32) - 127.5)/127.5
# X_train = X_train / 255 <-

X_train_n = X_train_n.reshape((X_train.shape[0],1,X_train.shape[1],X_train.shape[2]))
# new shape should be (60000,1,28,28) 1 channel image 28x28 px
X_train_n.shape

(60000, 1, 28, 28)

In [5]:
checkpoint_folder = 'chk/'
batch_size = 256
epochs = 100
batchs_per_epoch = int(X_train_n.shape[0]/batch_size)

In [6]:
# Instantiate Models

desc = keras_discriminator_model()
gen = keras_generator_model()
d_on_g = generator_and_discriminator(gen,desc)

d_op = SGD(lr = 5e-4,momentum = 0.9,nesterov=True)
g_op = SGD(lr = 5e-4,momentum = 0.9,nesterov=True)

gen.compile(loss='binary_crossentropy',optimizer = "SGD")

d_on_g.compile(loss='binary_crossentropy',optimizer = g_op)

desc.trainable = True # necessary??
desc.compile(loss='binary_crossentropy',optimizer= d_op)

In [7]:
noise = np.zeros((batch_size,100))

In [8]:
epoch_loss = []
for epoch in tqdm_notebook(range(epochs),desc = 'epoch'):
    batch_loss = []
    for batch_index in tqdm_notebook(range(batchs_per_epoch),desc = 'batch',leave=False):
        
        #build noise up
        for sample in range(batch_size):
            noise[sample,:] = np.random.uniform(-1,1,100)
        
        image_set = X_train_n[batch_index*batch_size:(batch_index+1)*batch_size]
        
        # feed noise into generator network
        gen_images = gen.predict(noise,verbose=0)

        # now we have data for our desc network
        X_desc = np.concatenate([image_set,gen_images],axis=0)
        y = [1] * batch_size + [0] * batch_size
        
        # train desc
        d_loss = desc.train_on_batch(X_desc,y)
        
        #build noise up
        for sample in range(batch_size):
            noise[sample,:] = np.random.uniform(-1,1,100)
            
        # train generator on improved desc
        desc.trainable = False
        g_loss = d_on_g.train_on_batch(noise,[1] * batch_size)
        desc.trainable = True
        
        if(batch_index+1 == batchs_per_epoch):
            gen.save_weights(checkpoint_folder+'g_weights.h5',overwrite=True)
            desc.save_weights(checkpoint_folder+'d_weights.h5',overwrite=True)
        batch_loss.append([d_loss,g_loss])
    
    avg_d = np.mean([x[0] for x in batch_loss])
    avg_g = np.mean([x[1] for x in batch_loss])
    print('avg desc loss {} || avg gen loss {}'.format(avg_d,avg_g))
    epoch_loss.append([avg_d,avg_g])

avg desc loss 0.0808791592717 || avg gen loss 3.05545806885
avg desc loss 0.0076539455913 || avg gen loss 4.38770008087
avg desc loss 0.00242618238553 || avg gen loss 5.41740655899
avg desc loss 0.00132587447297 || avg gen loss 5.99391412735
avg desc loss 0.000883951608557 || avg gen loss 6.38273286819
avg desc loss 0.00065121910302 || avg gen loss 6.6802945137
avg desc loss 0.00051131186774 || avg gen loss 6.91629648209
avg desc loss 0.000417206989368 || avg gen loss 7.11600780487
avg desc loss 0.000351458176738 || avg gen loss 7.28294897079
avg desc loss 0.000302845583064 || avg gen loss 7.42806386948



KeyboardInterrupt: 